In [ ]:
pip install -U scikit-learn plotly pyarrow

```python
# TODO: run it once finished to create the package requirements file
pip freeze > requirements.txt
# conda list -e > requirements.txt
```

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from datetime import datetime

Load the clean dataset created during data eng. steps:

In [4]:
df_listings = pd.read_parquet('data/listings_clean.parquet')

In [5]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)

In [6]:
df_listings.tail()

,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_maximum_nights,maximum_maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_shared_rooms,reviews_per_month,bathrooms_type,bathrooms_qty
20763,NaN,NaN,NaN,f,t,"Prahran East, Victoria, Australia",Port Phillip,-37.860111,145.003583,Entire home/apt,2,1.0,1.0,96,2,14,14,14,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,0,NaN,None,1
20764,within an hour,1.0,0.93,t,t,"St Kilda, Victoria, Australia",Port Phillip,-37.864430,144.988750,Entire home/apt,2,1.0,1.0,90,5,365,1125,1125,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4,0,NaN,None,1
20765,NaN,NaN,NaN,f,t,None,Melbourne,-37.784396,144.940760,Private room,2,1.0,1.0,55,1,365,365,365,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,0,NaN,private,1
20766,NaN,NaN,NaN,f,f,"St Kilda, Victoria, Australia",Port Phillip,-37.859512,144.978695,Entire home/apt,2,1.0,1.0,700,4,14,1125,1125,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1,0,NaN,None,1
20767,within an hour,1.0,0.91,f,t,"Carlton, Victoria, Australia",Melbourne,-37.804176,144.963635,Entire home/apt,4,1.0,2.0,122,1,365,365,365,0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,119,0,NaN,None,1


Checking dataset types:

In [7]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20768 entries, 0 to 20767
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   host_response_time                           13639 non-null  category      
 1   host_response_rate                           13639 non-null  float64       
 2   host_acceptance_rate                         14403 non-null  float64       
 3   host_is_superhost                            20759 non-null  category      
 4   host_identity_verified                       20766 non-null  object        
 5   neighbourhood                                12218 non-null  object        
 6   neighbourhood_cleansed                       20768 non-null  object        
 7   latitude                                     20768 non-null  float64       
 8   longitude                                    20768 non-null  float64       


TODO:
- Decide strategy for category attributes
- Explore which attributes would need normalization
- Decide regression algorithm
- How to include location?

Let's set the training strategy for the basemodel:

In [13]:
from sklearn.model_selection import train_test_split

Let's separate target attribute and the rest of dataset:

In [10]:
X = df_listings.drop('price', axis=1)
y = df_listings['price']

Split the dataset on train, validation and test sets (ratio ~70:20:10):

In [21]:
X_train, X_t, y_train, y_t = train_test_split(X, 
                                              y,
                                              test_size=0.3,
                                              random_state=99
                                              )

In [36]:
X_val, X_test, y_val, y_test = train_test_split(X_t, 
                                                y_t,
                                                test_size=0.3,
                                                random_state=99
                                              )

Now, let's use a Random Forest regressor for our baseline model, mainly because of simplicity and no need for feature scaling:

In [38]:
from sklearn.ensemble import RandomForestRegressor